In [ ]:
!pip install kaggle

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data = pd.read_csv('/content/twitter_validation.csv')

In [ ]:
twitter_data.shape

(999, 4)

In [ ]:
twitter_data.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [ ]:
twitter_data.shape

(999, 4)

In [ ]:
twitter_data.head()

,id,company,impact,review
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [ ]:
twitter_data.isnull().sum()

id         0
company    0
impact     0
review     0
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'impact' is a categorical column
label_encoder = LabelEncoder()
twitter_data['impact_numeric'] = label_encoder.fit_transform(twitter_data['impact'])


In [ ]:
twitter_data['impact'].value_counts()

Neutral       285
Positive      277
Negative      266
Irrelevant    172
Name: impact, dtype: int64

In [ ]:
twitter_data['impact_numeric'] = label_encoder.fit_transform(twitter_data['impact'])

In [ ]:
print(twitter_data['impact_numeric'])

0      0
1      2
2      1
3      1
4      2
      ..
995    0
996    0
997    3
998    3
999    2
Name: impact_numeric, Length: 1000, dtype: int64


In [ ]:
def stemming (content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  porter=PorterStemmer()
  stemmed_content = [porter.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['review'].apply(stemming)


In [ ]:
print(twitter_data['review'])

0      I mentioned on Facebook that I was struggling ...
1      BBC News - Amazon boss Jeff Bezos rejects clai...
2      @Microsoft Why do I pay for WORD when it funct...
3      CSGO matchmaking is so full of closet hacking,...
4      Now the President is slapping Americans in the...
                             ...                        
995    ⭐️ Toronto is the arts and culture capital of ...
996    tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997    Today sucked so it’s time to drink wine n play...
998    Bought a fraction of Microsoft today. Small wins.
999    Johnson & Johnson to stop selling talc baby po...
Name: review, Length: 1000, dtype: object


In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['impact_numeric'].values

In [ ]:
print(X)

['mention facebook struggl motiv go run day translat tom great aunti hayley get bed told grandma think lazi terribl person'
 'bbc news amazon boss jeff bezo reject claim compani act like drug dealer bbc co uk news av busin'
 'microsoft pay word function poorli samsungu chromebook'
 'csgo matchmak full closet hack truli aw game'
 'presid slap american face realli commit unlaw act acquitt discov googl vanityfair com news'
 'hi eahelp madelein mccann cellar past year littl sneaki thing escap whilst load fifa point took card use paypal account work help resolv pleas'
 'thank eamaddennfl new te austin hooper orang brown brown austinhoop pic twitter com grg xzfkon'
 'rocket leagu sea thiev rainbow six sieg love play three stream best stream twitch rocketleagu seaofthiev rainbowsixsieg follow'
 'ass still knee deep assassin creed odyssey way anytim soon lmao'
 'fix jesu pleas fix world go playstat askplayst playstationsup treyarch callofduti neg silver wolf error code pic twitter com ziryhrf 

In [ ]:
print(Y)

[0 2 1 1 2 1 3 3 3 1 3 3 1 2 1 3 3 1 3 1 1 2 0 1 2 2 1 0 0 1 3 3 1 3 1 2 2
 0 3 2 3 2 2 2 3 2 1 1 1 2 3 1 1 3 3 3 3 3 1 0 1 3 3 0 1 2 1 0 2 1 3 1 1 3
 3 0 3 0 2 2 2 3 3 2 3 2 1 0 1 2 2 1 3 0 0 1 1 1 2 3 2 1 3 3 2 3 2 3 1 2 2
 2 1 2 1 2 2 3 3 2 1 1 3 1 2 1 3 2 1 2 0 3 2 3 3 0 2 2 0 0 0 2 2 0 0 0 3 2
 3 0 3 1 2 2 2 0 2 1 2 3 1 2 1 0 0 0 2 1 1 1 3 3 3 2 2 3 0 2 2 2 3 2 1 1 2
 3 3 0 0 2 3 3 2 0 2 1 1 1 1 3 2 2 3 3 3 3 1 3 3 0 2 0 1 1 0 0 1 3 3 1 0 1
 3 3 1 0 0 3 3 1 3 0 2 0 0 1 2 2 3 1 0 0 3 3 0 0 2 3 1 1 3 3 3 3 2 2 3 1 2
 3 2 1 2 2 1 3 3 0 1 2 0 3 2 0 1 2 1 3 3 1 1 1 3 1 2 3 2 2 1 3 1 3 1 0 2 2
 3 1 2 1 0 3 1 3 0 3 3 3 3 3 1 1 3 1 2 2 2 3 0 2 3 0 1 2 2 0 2 2 0 1 3 1 1
 0 0 3 0 3 2 2 0 0 1 1 1 2 3 0 2 1 3 0 2 1 1 1 3 2 2 0 1 3 3 0 2 0 3 2 2 3
 3 1 2 3 1 2 1 0 1 3 3 0 3 3 2 1 2 0 0 3 2 3 1 1 1 0 3 2 3 0 1 2 0 1 3 3 3
 3 2 2 0 2 1 3 3 2 1 3 2 1 1 1 1 1 2 2 2 3 3 1 1 2 1 2 2 1 0 3 0 0 1 3 3 3
 0 2 3 0 2 2 1 3 2 2 2 2 0 0 0 2 2 0 2 0 2 0 1 1 3 0 0 3 0 0 2 1 1 3 3 1 3
 3 3 3 0 3 0 2 1 3 2 2 3 

In [ ]:
X_train,X_test,Y_train, Y_test=train_test_split(X,Y, test_size=0.2,stratify=Y, random_state=2)

In [ ]:
vectorizer= TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction= model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train, X_train_prediction)

In [ ]:
print(training_data_accuracy)

0.95125


In [ ]:
X_test_prediction= model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test, X_test_prediction)

In [ ]:
print(test_data_accuracy)

0.515
